Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

# LAB9

Write a local-search algorithm (eg. an EA) able to solve the *Problem* instances 1, 2, 5, and 10 on a 1000-loci genomes, using a minimum number of fitness calls. That's all.

### Deadlines:

* Submission: Sunday, December 3 ([CET](https://www.timeanddate.com/time/zones/cet))
* Reviews: Sunday, December 10 ([CET](https://www.timeanddate.com/time/zones/cet))

Notes:

* Reviews will be assigned  on Monday, December 4
* You need to commit in order to be selected as a reviewer (ie. better to commit an empty work than not to commit)

In [34]:
from random import choices, uniform
import random
from dataclasses import dataclass, field
from copy import deepcopy
from typing import Callable, List
import math
import numpy as np

import lab9.lab9_lib as lab9_lib

In [35]:
fitness = lab9_lib.make_problem(10)
for n in range(10):
    ind = choices([0, 1], k=50)
    print(f"{''.join(str(g) for g in ind)}: {fitness(ind):.2%}")

print(fitness.calls)

11111111001010000110110000100001001001001101001111: 23.36%
00010111101011110101001111100111011101001100110010: 9.11%
00101001000011010010100111100001101101010011001011: 7.33%
01101101010000111110111110110110111001001001101000: 9.13%
11100010001110111100111001110111010011100011101011: 9.11%
01111000010111111001001111001000101001010101000001: 31.36%
00101100101111010100111100010101001100110011111011: 15.33%
00011100111010100011001110111101011110110111110010: 19.11%
10011001001111111111111011001001101010101111111000: 9.11%
00000100011101110111110111001101011100011101000011: 9.11%
10


# Individual
It defines the individual of the ES strategy, it is composed by a genome, which is the binary representations with length n:loci an by a fitness score, which is computed using the callable fitness_function passed as parameter
# Operator Agent
It defines an agent which compute the variation operators and has a global view on the statistics of the operator applied, initially the purpose was to dinamically adapt the parameter that regolates the use of the operators and the parameters, based on the success statistics of the operators (child with a fitness score higher than the parents)

In [36]:
@dataclass
class Individual:
    genome : tuple[int]
    n_loci: int
    fitness: float
    fitness_func: Callable[[tuple[int]], float]
    distance: float
    
    def __init__(self, FITNESS_FUNC, n_loci, genome = None):
        self.n_loci = n_loci
        if genome is None:
            self.genome = random.choices([0, 1], k=n_loci)
        else:
            self.genome = genome
        self.fitness_func = FITNESS_FUNC
        self.fitness = self.fitness_func(self.genome)
    
    def __repr__(self):
        return f"{[str(val) for val in self.genome]}"
    
@dataclass
class Operators_agent:
    
    n_points_number: int
    mutation_points: int
    points_to_mutate: int
    mutation_rate: float
    crossover_functions : []

    
    def __init__(self, n_points_number, points_to_mutate, mutation_rate):
        self.n_points_number = n_points_number
        self.mutation_points = 1
        self.points_to_mutate = points_to_mutate
        self.mutation_rate = mutation_rate
        self.crossover_functions = [Operators_agent.crossover_one_point, Operators_agent.crossover_n_point, Operators_agent.crossover_uniform]
        
    
    def mutate(self, individual: "Individual"):
        
        index_to_mutate = random.choices(range(individual.n_loci), k=self.points_to_mutate)
        new_genome = deepcopy(individual.genome)
        for i in index_to_mutate:
            new_genome[i] = 1 - new_genome[i]
        return Individual(individual.fitness_func, individual.n_loci, new_genome)
    
    def crossover(self, ind1: "Individual", ind2: "Individual", cross_operation: int):
        crossover_function = self.crossover_functions[cross_operation]
        child = crossover_function(ind1, ind2, self.n_points_number)
        return child
    
    def crossover_one_point(ind1: "Individual", ind2: "Individual", n):
        index_to_cross = random.randrange(ind1.n_loci)
        child = Individual(ind1.fitness_func, ind1.n_loci, ind1.genome[:index_to_cross] + ind2.genome[index_to_cross:])
        return child
        
    def crossover_n_point(ind1: "Individual", ind2: "Individual", n):
        index_list = sorted(random.sample(range(ind1.n_loci), n))
        child1_genome, child2_genome = ind1.genome[:], ind2.genome[:]
        for i in range(0, n, 2):
            start = index_list[i]
            end = index_list[i+1] if i+1 < n else len(ind1.genome)
            child1_genome[start:end] = child2_genome[start:end]
        return Individual(ind1.fitness_func, ind1.n_loci, child1_genome)
                    
    def crossover_uniform(ind1: "Individual", ind2: "Individual", n):
        index_list = [random.random() for _ in range(ind1.n_loci)]
        return Individual(ind1.fitness_func, ind1.n_loci, [i1 if index < .5 else i2 for i1, i2, index in zip(ind1.genome, ind2.genome, index_list)])

# Parent selection function
This function compute a stochastic universal sampling for parent selection and use as parameter the reduction factor "parent_selection_rate" to indicates how much to reduce the initial populations
This technique is an alternative to the roulette wheel algorithm, conceptually it is equivalent to making one spin of a wheel with lambda equally spaced arms rather than lambda spins of a one armed wheel like the roulette, and that make a better sample of the distribution.
The resulting population will be the one where the tournament takes place

In [37]:
def stochastic_universal_sampling(population: List[Individual], parent_selection_rate: int) -> List[Individual]:
        
    total_fitness = sum(ind.fitness for ind in population)
    pointer_distance = total_fitness / len(population)
    num_selected_parents = int(parent_selection_rate)
    start = uniform(0, pointer_distance)
    pointers = [start + i * pointer_distance for i in range(num_selected_parents)]

    new_population = []
    current_index = 0
    for pointer in pointers:
        while pointer > 0:
            pointer -= population[current_index].fitness
            current_index = (current_index + 1) % len(population)
        new_population.append(population[current_index])
        if len(new_population) == num_selected_parents:
            break 

    return new_population

# Tournament function
It implements a tournament selection to take a parent to use for recombination or reproduction

In [38]:
def select_parent(parents: list[Individual], size: int):
    pool = choices(parents, k=size)
    winner = max(pool, key=lambda ind: ind.fitness)
    return winner

# Offspring generation function
It generates the offspring by apply, using the tournament function to select the parent each iteration, a recombination or mutation depending on the mutation rate

In [39]:
def offspring_generation(parents: List[Individual], offspring_size: int, operator_agent: "Operators_agent", tournament_size: int, cross_operation) -> List[Individual]:
    offspring = []
    offspring_length = int(offspring_size)
    
    for _ in range(offspring_length):
        p = select_parent(parents, tournament_size)
        if random.random() < operator_agent.mutation_rate:
            offspring.append(operator_agent.mutate(p))
        else:
            p2 = select_parent(parents, tournament_size)
            offspring.append(operator_agent.crossover(p, p2, cross_operation))
                  
    return offspring

In [69]:
def offspring_generation_mixture(parents: List[Individual], offspring_size: int, operator_agent: "Operators_agent", tournament_size: int, cross_operation) -> List[Individual]:
    offspring = []
    offspring_length = int(offspring_size)
    
    for _ in range(offspring_length):
        p = select_parent(parents, tournament_size)
        p2 = select_parent(parents, tournament_size)
        if random.random() < operator_agent.mutation_rate:
            offspring.append(operator_agent.mutate(operator_agent.crossover(p, p2, cross_operation)))
        else:
            
            offspring.append(operator_agent.crossover(p, p2, cross_operation))
                  
    return offspring

In [50]:
@dataclass
class Island:
    population: List[Individual]
    
    def __init__(self, POPULATION_SIZE: int, fitness_func: Callable[[tuple[int]], float], n_loci: int):
        self.population = []
        for _ in range(POPULATION_SIZE):
            self.population.append(Individual(fitness_func, n_loci))
    
    def take_best(self):
        return max(self.population, key=lambda ind: ind.fitness)
            
def move_migrants(island1: "Island", island2: "Island",  n_migrants: int, distance_function) -> List[Individual]:
    # Calculate distances from island1 to island2
    distances_1_to_2 = [sum(distance_function(ind1, ind2) for ind2 in island2.population) / len(island2.population)
                        for ind1 in island1.population]

    # Calculate distances from island2 to island1
    distances_2_to_1 = [sum(distance_function(ind2, ind1) for ind2 in island1.population) / len(island1.population)
                        for ind1 in island2.population]

    # Sort indices based on distances
    island1_sorted_indices = sorted(range(len(island1.population)), key=lambda i: distances_1_to_2[i], reverse=True)
    island2_sorted_indices = sorted(range(len(island2.population)), key=lambda i: distances_2_to_1[i], reverse=True)

    # Select best individuals as migrants
    best_ind_from_island_1 = [island1.population[idx] for idx in island1_sorted_indices[:n_migrants]]
    best_ind_from_island_2 = [island2.population[idx] for idx in island2_sorted_indices[:n_migrants]]

    # Update populations by removing selected migrants and adding new migrants
    island1.population = [ind for idx, ind in enumerate(island1.population) if idx >= n_migrants]
    island2.population = [ind for idx, ind in enumerate(island2.population) if idx >= n_migrants]

    island1.population.extend(best_ind_from_island_2)
    island2.population.extend(best_ind_from_island_1)
        

    

# Distance function

In [41]:
import numpy as np

def hamming_distance(ind1: "Individual", ind2: "Individual") -> float:
    distance = 0
    for i in range(ind1.n_loci):
        if ind1.genome[i]!= ind2.genome[i]:
            distance += 1
    return distance/len(ind1.genome)

def jaccard_distance(ind1: "Individual", ind2: "Individual") -> float:
    intersection = np.logical_and(ind1.genome, ind2.genome).sum();
    union = np.logical_or(ind1.genome, ind2.genome).sum();
    return 1-(intersection/union)

def euclidean_distance(ind1: "Individual", ind2: "Individual") -> float:
    distance = sum((int(ind1.genome[i]) - int(ind2.genome[i]))**2 for i in range(ind1.n_loci))
    max_possible_distance = (2 ** 0.5) * ind1.n_loci  # Maximum possible distance for normalized Euclidean distance
    normalized_distance = (distance ** 0.5) / max_possible_distance
    return normalized_distance

def manhattan_distance_binary(ind1: "Individual", ind2: "Individual") -> float:
    
    distance = sum(abs(int(ind1.genome[i]) - int(ind2.genome[i])) for i in range(ind1.n_loci))
    max_possible_distance = ind1.n_loci  # Maximum possible distance for normalized Manhattan distance
    normalized_distance = distance / max_possible_distance
    return normalized_distance
    

# Evolutionary algorithm
It implements all the evolutionary cycle
The principal steps are:
* Initialization
    * Initializes the fitness_function with the specified instance
    * Generates the initial population randomically
* For each generation:
    * It first applies elitism to select the best parent
    * It generates the offspring population, with a size specifying by the offspring rate
    * It selects the best individual considering the union of both selected parents and offspring or just the offspring, depending on the selection type
    * It checks if the best individual has reached the fitness score goal
* to compute the variation operators it uses an operator agent
    
* In the end it returns both the best individual and the number of fitness calls 

In [42]:


def launch_es_cycle(problem_instance: int, populations_number: int, n_loci: int, generations: int, offspring_size: int, tournament_size: int, selection_type: int, operator_agent: "Operators_agent", cross_operation: int):
    
    fitness_func = lab9_lib.make_problem(problem_instance)
    population = [Individual(fitness_func, n_loci) for _ in range(populations_number)]
    
    for gen in range(generations):
        
        offspring = offspring_generation(population, offspring_size, operator_agent, tournament_size, cross_operation)
        population = sorted(population+offspring if selection_type == 1 else offspring, key=lambda p: p.fitness, reverse=True)[:populations_number]
        if math.isclose(1, population[0].fitness):
            break
        
    return population[0], fitness_func.calls

# Parameters tuning
This code has the purpose to tune some parameters, considering a small populations size and generations number, to find the best combination that will be used then to test the algorithm
The parameters are:
* OFFSPRING_SIZE: It indicates the size of the generated offspring
* CROSS_OPERATIONS: It contains an array of 3 integer, each one corresponding to a different cross function
* NUMBER_CROSSOVER_POINTS: It contains an array of integer that indicates the point in which the parents will be split in the crossover_n_point function
* TOURNAMENT_SIZE: It indicates how much is the size of the array of individuals selected to compete in the tournament selection
* MUTATION_POINTS: It indicates how much points the mutation change

In [919]:
import itertools

LOCI = 1000
PROBLEM_INSTANCES = [1, 2, 5, 10]

GENERATIONS_TEST = 1000
POPULATIONS_NUMBER_TEST = 50

OFFSPRING_SIZE = 20
NUMBER_CROSSOVER_POINTS = 2
MUTATION_RATES = {1: 0.8, 2: 0.2, 5: 0.2, 10: 0.2}

TOURNAMENT_SIZE = [2, 4, 8]
SELECTION_TYPE = [0, 1]
CROSS_OPERATIONS = [0, 1, 2]
MUTATION_POINTS = [1, 4]

parameter_combinations = list(itertools.product(PROBLEM_INSTANCES, TOURNAMENT_SIZE, SELECTION_TYPE, MUTATION_POINTS, CROSS_OPERATIONS))

best_parameters = {}
best_individuals = {}

for instance, t_size, s_type, m_points, cr_op in parameter_combinations:
    operator_agent = Operators_agent(NUMBER_CROSSOVER_POINTS, m_points, MUTATION_RATES[instance])
    individual, calls = launch_es_cycle(instance, POPULATIONS_NUMBER_TEST, LOCI, GENERATIONS_TEST, OFFSPRING_SIZE, t_size, s_type, operator_agent, cr_op)
    if instance not in best_parameters or (individual.fitness > best_individuals[instance][0].fitness)\
            or (math.isclose(individual.fitness, best_individuals[instance][0].fitness) and calls < best_individuals[instance][1]):
        best_individuals[instance] = [individual, calls]
        best_parameters[instance] = [t_size, s_type, m_points, cr_op]

for instance in PROBLEM_INSTANCES:
    print(f"Instance {instance}\n "
          f"Best Individual: Score: {best_individuals[instance][0].fitness}\n "              
          f"Calls: {best_individuals[instance][1]}")
    print(f"Parameters:\n "
          f"Tournament size:{best_parameters[instance][0]}\n"
          f"Selection type:{best_parameters[instance][1]}\n"
          f"Points to mutate:{best_parameters[instance][2]}\n"
          f"Cross operation:{best_parameters[instance][3]}\n")


Instance 1
 Best Individual: Score: 1.0
 Calls: 17390
Parameters:
 Tournament size:8
Selection type:1
Points to mutate:1
Cross operation:2

Instance 2
 Best Individual: Score: 0.812
 Calls: 20050
Parameters:
 Tournament size:8
Selection type:1
Points to mutate:4
Cross operation:2

Instance 5
 Best Individual: Score: 0.4656
 Calls: 20050
Parameters:
 Tournament size:4
Selection type:1
Points to mutate:4
Cross operation:1

Instance 10
 Best Individual: Score: 0.40372
 Calls: 20050
Parameters:
 Tournament size:4
Selection type:1
Points to mutate:4
Cross operation:0


# Instance 1 
Parameter tuning for offspring and population size


In [921]:
TOURNAMENT_SIZE = 8
SELECTION_TYPE = 1
POINTS_TO_MUTATE = 1
CROSS_OPERATION = 2
INSTANCE = 1
POPULATION_SIZE = [20, 40, 60, 80]
OFFSPRING_SIZE = [20, 30, 40, 50]
MUTATION_RATE = 0.8
GENERATIONS = 5000

parameter_combinations = list(itertools.product(POPULATION_SIZE, OFFSPRING_SIZE))

best_parameters = {}
best_individuals = {}

for p_size, of_size in parameter_combinations:
    operator_agent = Operators_agent(INSTANCE, POINTS_TO_MUTATE, MUTATION_RATE)
    individual, calls = launch_es_cycle(INSTANCE, p_size, LOCI, GENERATIONS, of_size, TOURNAMENT_SIZE, SELECTION_TYPE, operator_agent, CROSS_OPERATION)
    if instance not in best_parameters or (individual.fitness > best_individuals[instance][0].fitness)\
            or (math.isclose(individual.fitness, best_individuals[instance][0].fitness) and calls < best_individuals[instance][1]):
        best_individuals[instance] = [individual, calls]
        best_parameters[instance] = [p_size, of_size]

print(f"Instance {INSTANCE}\n "
          f"Best Individual: Score: {best_individuals[instance][0].fitness}\n "              
          f"Calls: {best_individuals[instance][1]}")
print(f"Parameters:\n "
          f"Population size:{best_parameters[instance][0]}\n "
          f"Offspring size:{best_parameters[instance][1]}\n")

Instance 1
 Best Individual: Score: 1.0
 Calls: 11660
Parameters:
 Population size:20
Offspring size:20


This function enhanced the previous one by adding a control on the standard deviation and, in case of a too low value, it decrement the population to a minor number with the stochastic universal sampling and then add random individual to reobtain the original population number

In [46]:
import numpy as np

def launch_es_cycle_with_survival_selection(problem_instance: int, populations_number: int, n_loci: int, generations: int, offspring_size: int, tournament_size: int, selection_type: int, operator_agent: "Operators_agent", cross_operation: int, parent_selection_size: int):
    
    fitness_func = lab9_lib.make_problem(problem_instance)
    population = [Individual(fitness_func, n_loci) for _ in range(populations_number)]
    
    for gen in range(generations):
        
        offspring = offspring_generation(population, offspring_size, operator_agent, tournament_size, cross_operation)
        population = sorted(population+offspring if selection_type == 1 else offspring, key=lambda p: p.fitness, reverse=True)[:populations_number]
        if math.isclose(1, population[0].fitness):
            break
        if np.std([p.fitness for p in population]) < 0.005:
            best_individual = population[0]
            population.remove(population[0])
            population = stochastic_universal_sampling(population, parent_selection_size)
            population.append(best_individual)
            population.extend([Individual(fitness_func, n_loci) for _ in range(populations_number-len(population))])
        
    return population[0], fitness_func.calls

In [943]:
LOCI = 1000
PROBLEM_INSTANCES = [2, 5, 10]

TOURNAMENT_SIZE = {2: 8, 5: 4, 10: 4}
SELECTION_TYPE = 1
POINTS_TO_MUTATE = 4
CROSS_OPERATION = {2: 2, 5: 1, 10: 0}
NUMBER_CROSSOVER_POINTS = 2
PARENT_SELECTION_SIZE = [35, 65]
POPULATION_SIZE = 100
OFFSPRING_SIZE = 50
MUTATION_RATE = [0.2, 0.6]
GENERATIONS = 40000


parameter_combinations = list(itertools.product(PROBLEM_INSTANCES, PARENT_SELECTION_SIZE, MUTATION_RATE))

best_parameters = {}
best_individuals = {}

for instance, p_s_s, m_rate in parameter_combinations:
    operator_agent = Operators_agent(NUMBER_CROSSOVER_POINTS, POINTS_TO_MUTATE, m_rate)
    individual, calls = launch_es_cycle_with_survival_selection(instance, POPULATION_SIZE, LOCI, GENERATIONS, OFFSPRING_SIZE, TOURNAMENT_SIZE[instance], SELECTION_TYPE, operator_agent, CROSS_OPERATION[instance], p_s_s)
    if instance not in best_parameters or (individual.fitness > best_individuals[instance][0].fitness)\
            or (math.isclose(individual.fitness, best_individuals[instance][0].fitness) and calls < best_individuals[instance][1]):
        best_individuals[instance] = [individual, calls]
        best_parameters[instance] = [p_s_s, m_rate]

for instance in PROBLEM_INSTANCES:
    print(f"Instance {instance}\n "
          f"Best Individual: Score: {best_individuals[instance][0].fitness}\n "              
          f"Calls: {best_individuals[instance][1]}")
    print(f"Parameters:\n "
          f"Parent Selection Rate:{best_parameters[instance][0]}\n"
          f"Mutation Rate:{best_parameters[instance][1]}\n")

Instance 2
 Best Individual: Score: 0.998
 Calls: 2776388
Parameters:
 Parent Selection Rate:65
Mutation Rate:0.6

Instance 5
 Best Individual: Score: 0.72
 Calls: 2650248
Parameters:
 Parent Selection Rate:65
Mutation Rate:0.6

Instance 10
 Best Individual: Score: 0.4
 Calls: 2634532
Parameters:
 Parent Selection Rate:35
Mutation Rate:0.6


In [70]:

def launch_islands_model(problem_instance: int, populations_number_per_islands: int, islands_number: int, migrants_number: int, n_loci: int, generations: int, offspring_size: int, tournament_size: int, selection_type: int, operator_agent: "Operators_agent", cross_operation: int, parent_selection_size: int, distance_function):
    
    fitness_func = lab9_lib.make_problem(problem_instance)
    islands_list = [Island(populations_number_per_islands, fitness_func, n_loci) for _ in range(islands_number)]
            
    for gen in range(generations):
        if (gen % 5 == 0):
            for _ in range(islands_number//2):
                index_1 = np.random.randint(0, islands_number)
                index_2 = np.random.randint(0, islands_number)
                while index_1 == index_2:
                    index_2 = np.random.randint(0, islands_number)
                island1, island2 = islands_list[index_1], islands_list[index_2]
                move_migrants(island1, island2, migrants_number, distance_function)
        for island in islands_list:
            offspring = offspring_generation_mixture(island.population, offspring_size, operator_agent, tournament_size, cross_operation)
            island.population = sorted(island.population+offspring if selection_type == 1 else offspring, key=lambda p: p.fitness, reverse=True)[:populations_number_per_islands]
            if math.isclose(1, island.population[0].fitness):
                break
            if np.std([p.fitness for p in island.population]) < 0.005:
                best_individual = island.population[0]
                island.population.remove(island.population[0])
                island.population = stochastic_universal_sampling(island.population, parent_selection_size)
                island.population.append(best_individual)
                island.population.extend([Individual(fitness_func, n_loci) for _ in range(populations_number_per_islands-len(island.population))])
        best = None
        for island in islands_list:
            best_of_island = island.take_best()
            if best is None or (best_of_island.fitness > best.fitness):
                best = best_of_island
        
        
    return best, fitness_func.calls
    

In [72]:
import itertools

LOCI = 1000
PROBLEM_INSTANCES = [2, 5, 10]

TOURNAMENT_SIZE = {2: 8, 5: 4, 10: 4}
SELECTION_TYPE = 1
POINTS_TO_MUTATE = 4
CROSS_OPERATION = {2: 2, 5: 1, 10: 0}
NUMBER_CROSSOVER_POINTS = 2
PARENT_SELECTION_SIZE = {2: 65, 5: 65, 10: 35}
OFFSPRING_SIZE = 50
MUTATION_RATE = 0.6
ISLANDS_NUMBER = 5
POPULATION_SIZE_PER_ISLANDS = 40
MIGRANTS_NUMBER = 10
DISTANCE_FUNCTION = [hamming_distance]
GENERATIONS = 10000


best_individuals = {2: None, 5: None, 10: None}

for instance in PROBLEM_INSTANCES:
    operator_agent = Operators_agent(NUMBER_CROSSOVER_POINTS, POINTS_TO_MUTATE, MUTATION_RATE)
    individual, calls = launch_islands_model(instance, POPULATION_SIZE_PER_ISLANDS, ISLANDS_NUMBER, MIGRANTS_NUMBER, LOCI, GENERATIONS, OFFSPRING_SIZE, TOURNAMENT_SIZE[instance], SELECTION_TYPE, operator_agent, CROSS_OPERATION[instance], PARENT_SELECTION_SIZE[instance], hamming_distance)
    if  best_individuals[instance] == None or (individual.fitness > best_individuals[instance][0].fitness)\
            or (math.isclose(individual.fitness, best_individuals[instance][0].fitness) and calls < best_individuals[instance][1]):
        best_individuals[instance] = [individual, calls]
        

for instance in PROBLEM_INSTANCES:
    print(f"Instance {instance}\n "
          f"Best Individual: Score: {best_individuals[instance][0].fitness}\n "              
          f"Calls: {best_individuals[instance][1]}")          

Instance 2
 Best Individual: Score: 0.99
 Calls: 3999299
Instance 5
 Best Individual: Score: 0.515
 Calls: 3999340
Instance 10
 Best Individual: Score: 0.33
 Calls: 4199537


In [55]:
operator_agent = Operators_agent(2, 4, 0.6)
individual, calls = launch_islands_model(5, 50, 20, 20, 1000, 1000, 40, 4, 1, operator_agent, 1, 65, euclidean_distance)
print(f"Best Individual: Score: {individual.fitness}\n Calls: {calls}")

Best Individual: Score: 0.55
 Calls: 801000


In [56]:
operator_agent = Operators_agent(2, 4, 0.6)
individual, calls = launch_islands_model(5, 50, 20, 20, 1000, 1000, 40, 4, 1, operator_agent, 1, 65, manhattan_distance_binary)
print(f"Best Individual: Score: {individual.fitness}\n Calls: {calls}")

Best Individual: Score: 0.6143
 Calls: 801000


In [57]:
operator_agent = Operators_agent(2, 4, 0.6)
individual, calls = launch_islands_model(5, 50, 20, 20, 1000, 1000, 40, 4, 1, operator_agent, 1, 65, jaccard_distance)
print(f"Best Individual: Score: {individual.fitness}\n Calls: {calls}")

Best Individual: Score: 0.6063
 Calls: 801000


In [60]:
operator_agent = Operators_agent(2, 4, 0.6)
individual, calls = launch_islands_model(5, 50, 20, 20, 1000, 1000, 40, 4, 1, operator_agent, 1, 65, hamming_distance)
print(f"Best Individual: Score: {individual.fitness}\n Calls: {calls}")

Best Individual: Score: 0.6309
 Calls: 801000
